In [14]:
import ast

import datasets
import jsonlines as jsonl
import numpy as np
import rich
import transformers
import torch
import wget


In [10]:
# download dataset
url = "https://raw.githubusercontent.com/google-research/google-research/master/mbpp/mbpp.jsonl"
output_file = "mbpp.jsonl"
wget.download(url, output_file)
with open(output_file, "r") as f:
    reader = jsonl.Reader(f)
    data = list(reader)


In [79]:
def prep():
    samples = []
    lengths = []
    for entry in data:
        if entry["test_setup_code"]:
            rich.print("[red]Skipping because there is setup code")
            continue

        assert len(entry["test_list"]) == 3, len(entry["test_list"])
        
        for code in entry["test_list"]:
            parsed = ast.parse(code)
            assert len(parsed.body) == 1, len(parsed.body)
            assert_obj = parsed.body[0]
            assert isinstance(assert_obj, ast.Assert), type(assert_obj)
            assert isinstance(assert_obj.test.left, ast.Call), type(assert_obj.test.left)
            call_obj = assert_obj.test.left
            call_args = call_obj.args
            output_constant_obj = assert_obj.test.comparators
            assert len(output_constant_obj) == 1, len(output_constant_obj)
            
            node = output_constant_obj[0]
            if isinstance(node, ast.Call):
                print(code)
                rich.print("[red] Skipping because Call in comparator")
                continue

            if isinstance(node, ast.Name):
                print(code)
                rich.print("[red] Skipping because Name in comparator")
                continue

            output_constant_val = ast.literal_eval(node)
            args_vals = []
            ignore_case = False
            for arg in call_args:
                result = None

                if isinstance(arg, ast.Name):
                    print(code)
                    rich.print("[red]Ignored because there was a name in the args of the fn")
                    ignore_case = True 
                    break

                if isinstance(arg, ast.Call):
                    print(code)
                    rich.print("[red]Ignored because there was a call in the args of the fn")
                    ignore_case = True 
                    break

                if isinstance(arg, ast.BinOp):
                    left = ast.literal_eval(arg.left)
                    right = ast.literal_eval(arg.right)
                    if isinstance(arg.op, ast.Add):
                        result = left + right
                    elif isinstance(arg.op, ast.Sub):
                        result = left - right
                    elif isinstance(arg.op, ast.Mult):
                        result = left * right
                    elif isinstance(arg.op, ast.Div):
                        result = left / right
                    else:
                        raise NotImplementedError(type(arg.op))

                if result is None:
                    try:                    
                        parsed_arg = ast.literal_eval(arg)
                    except ValueError as err:
                        print(code)
                        rich.print("[red]Ignored because an arg wasn't a litteral")
                        ignore_case = True 
                        break
                else:
                    parsed_arg = result

                args_vals.append(parsed_arg)

            if not ignore_case:
                lengths.append(len(entry["code"].strip().split("\n")))
                samples.append((args_vals, output_constant_val))

    return samples, lengths

samples, lengths = prep() 
print()
print(len(samples))
print(len(data) * 3)

Skipping because there is setup code

assert remove_datatype((4, 5, 4, 7.7, 1.2), int) == [7.7, 1.2]


Ignored because there was a name in the args of the fn

assert remove_datatype((7, 8, 9, "SR"), str) == [7, 8, 9]


Ignored because there was a name in the args of the fn

...

Ignored because there was a call in the args of the fn

assert int(lobb_num(4, 2)) == 20


Ignored because there was a call in the args of the fn

Skipping because there is setup code

2904
2922
